# Casusopdracht - Films

**Klas:** V2B
**Studenten:** 
- Roan Gaasbeek
- Mathijs
- Luc

### Inhoud ###
- [Onderzoeksvragen](1.-Onderzoeksvragen)

In [35]:
import numpy as np
import pandas as pd

# 1. Onderzoeksvragen
De volgende onderzoeksvragen worden behandeld in deze opdracht:

- In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op $Facebook$ en $IMDB$?
- ** zelf nog een vraag bedenken met betrekking op een zelf bijgezochte dataset **
- In hoeverre is het mogelijk om logische clusters te vinden o.b.v. onder andere budget en omzet? Denk aan blockbusters (hoog budget en hoge omzet), flops (hoog budget en lage omzet) of cultfilms (laag budget en hoge omzet). Je mag extra features toevoegen. 

# 2. Data understanding

De eesrte stap in deze opdracht is de data understanding fase van $CRISP-DM$ dit houdt in:
- Data Collection
- Data Exploration & Analysis

#### 2.1 Data Collection
- de meegekregen movie.csv
- nog een dataset erbij zoeken

In [36]:
# Dit is de casus dataset
dataset = pd.read_csv('movie.csv')

# dit is de gekozen externe dataset


#### 2.2 Data Exploration & Analysis

In [ ]:
# nu gebruiken we deze functie om een eerste blik van de dataset te krijgen
dataset.head()

In [ ]:
# dit zijn de features met de datatypen
dataset.dtypes

met de volgende functie krijgen we een mooie samenvatting van de data

In [ ]:
dataset.describe()

een paar dingen die opvallen zijn:
- veel data die een minimum waarde van 0 hebben zijn dit missende waardes of daadwerkelijk 0?
- 

### Analyse van feature- en targetvariabelen

Te onderzoeken featurevariabelen:  
- movie_facebook_likes
- imdb_score
- budget

Te onderzoeken targetvariabele:  
- gross

In [ ]:
# Kolommen selecteren
kolommen = ['movie_facebook_likes', 'imdb_score', 'budget', 'gross']
data_analyse = dataset[kolommen]
data_analyse


In [ ]:
# Centrum- en spreidingsmaten
data_analyse.describe()

### Waarnemingen centrum- en spreidingsmaten

- De kolommen movie_facebook_likes en imdb_score hebben beide 5043 waarden, maar budget en gross hebben minder waarden (4551 en 4159 waarden).  
We zullen hier een oplossing voor moeten vinden, door bijvoorbeeld de rijen met missende waarden te droppen van de dataset.
- Q1 voor movie_facebook_likes is 0, dit betekent dat ten minste 25% van de films 0 facebook likes hebben. Dit komt waarschijnlijk omdat deze films geen facebook-pagina hebben.


In [ ]:
# Uitschieters bepalen
for kolom in data_analyse:

    # Q1, Q3 en IQR bepalen
    q1 = data_analyse[kolom].quantile(0.25)
    q3 = data_analyse[kolom].quantile(0.75)
    iqr = q3 - q1

    # Limieten uitschieters berekenen
    klein_laag = q1 - 1.5 * iqr
    klein_hoog = q3 + 1.5 * iqr
    extreem_laag = q1 - 3 * iqr
    extreem_hoog = q3 + 3 * iqr

    print(data_analyse[kolom].name)

    # Kleine uitschieters (1.5 - 3 IQR)
    kleine_uitschieters = data_analyse[kolom][((data_analyse[kolom] < klein_laag) & (data_analyse[kolom] > extreem_laag)) | ((data_analyse[kolom] > klein_hoog) & (data_analyse[kolom] < extreem_hoog))]
    print(f"Kleine uitschieters: {kleine_uitschieters.count()}")
 
    # Extreme uitschieters (3+ IQR)
    extreme_uitschieters = data_analyse[kolom][(data_analyse[kolom] < extreem_laag) | (data_analyse[kolom] > extreem_hoog)]
    print(f"Extreme uitschieters: {extreme_uitschieters.count()}")


### Waarnemingen

- movie_facebook_likes heeft 930 extreme uitschieters, dat is behoorlijk veel. Zoals eerder benoemd, komt dit waarschijnlijk doordat er veel films zijn zonder facebook pagina, wat geregistreerd wordt als 0 likes.
- imdb_score heeft 0 extreme uitschieters. Dit lijkt ons logisch omdat de imdb score voor een film altijd tussen de 1 en 10 ligt. Er zal dus nooit een waarde van bijvoorbeeld 100 moeten zijn, wat een extreme uitschieter zou zijn.

# 3. Data preparation
De volgende stap in deze is de data preparation fase van $CRISP-DM$ dit houdt in:
- Data Cleaning
- Data Processing

#### 3.1 Data Processing

{hier kunnen we colummen die we niet nodig hebben verwijderen en de volgorde aanpassen}


In [ ]:
dataset.drop(columns=['color','director_name','director_facebook_likes','actor_1_name', 'actor_1_facebook_likes','actor_2_name', 'actor_2_facebook_likes', 'actor_3_name', 'actor_3_facebook_likes','duration','genres','num_voted_users','cast_total_facebook_likes','facenumber_in_poster','plot_keywords', 'num_user_for_reviews','num_critic_for_reviews', 'language', 'country', 'content_rating', 'title_year','aspect_ratio'], inplace=True)

dataset.rename(columns={'movie_title':'title'}, inplace=True)

dataset.head()

** mischien dat we de facebook likes willen bewaren maar wilde gewoon wat proberen **

#### 3.2 Data Cleaning

